In [1]:
#imports
import csv
import time
import math 
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Float, String, MetaData
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from urllib.parse import urlparse
from datetime import datetime
import re
import os

In [2]:
# testing area
#initial testing with a single page before using loops later on. Using soup to find and scrape the values I was looking for 

# browser = Browser('edge')
#url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=20"
#browser.visit(url)
#html = browser.html
#html_soup = soup(html, 'html.parser')

In [3]:
# testing area 

# product_info = html_soup.find_all(class_='nl-product__content')
# product_data1 = []

# for product in product_info:
#     product_name_element = product.find('div', class_='nl-product-card__title')
#     original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
#     clearance_price_element = product.find('span', class_='nl-price--total--red')
#     rating_element = product.find('div', class_='bv_text')
#     image_element = product.find('div', class_='nl-product-card__image-wrap')
#     anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
#     img_tag = product.find('img', attrs={'data-src': True})

#     product_name = product_name_element.text.strip() if product_name_element else 'N/A'
#     original_price = original_price_element.text.strip() if original_price_element else 'N/A'
#     clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
#     rating = rating_element.text.strip() if rating_element else None
#     product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
#     product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
#     product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
#     product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

#     if original_price == 'N/A':
#         alt_original_price_element = product.find('span', class_='nl-price--total')
#         original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

#     clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
#     original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

#     product_data1.append({
#         'Product Name': product_name,
#         'Original Price': original_price,
#         'Clearance Price': clearance_price,
#         'Rating': rating,
#         'Product Code': product_code,
#         'Link': product_link, 
#         'Image Link': product_image_link
#     })

# print(product_data1)

In [4]:
# testing area 

# df1 = pd.DataFrame(product_data1)
# df1.sample(24)
# df1['Percentage Off'] = round(((df1['Original Price'] - df1['Clearance Price']) / df1['Original Price']) * 100)
# df1.sample(24)

In [5]:
# testing area 

#setup urlparse to grab the product category from the image link. 

# def extract_category(link_url):
#     parsed_url = urlparse(link_url)
#     path_components = parsed_url.path.split('/')
#     category = '/'.join(path_components[3:4])
#     return category

# df1['Product Category'] = df1['Image Link'].apply(extract_category)

# df1

In [6]:
#establishing the base url from the clearance section

base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

#setting up some options for the chrome browser. Using the exp_option to click the allow location, and incognito mode to
#seperate the browers from my other open browsers. Also, repeat testing without cookies saved was helpful. 

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

#setup the Chromedriver. it was not working without the time delay, perhaps letting the page load fully.
service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(base_url)
time.sleep(5)
html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')


# I wanted to dynamically determine the number of loops to use. I judged that total results, or total number of 
# products on sale, divided by the default number of products per page worked well. Using math.ceil to always round up. 

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'
total_results = int(re.search(r'\d+', total_results_text).group())
items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)


# creating a empty list to store data
product_data = []


#beginning the loop, adding time for the page to load, using a try except to handle a random survey pop-up. 

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

#gathering the ingredients for my soup. 

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(class_='nl-product__content')

#another loop to grab all the info on the page. The first code chunk locates, the second chunk parses out. 
    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

#there were some products with prices in different places, so this fills in those spaces. 
        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

#adding some data cleaning and parsing to the loop to make it easier later on. 
        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver.quit()

#printing these as a way to check that the scraping was working and how many deals there were
print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 1146
Total number of pages: 48


In [7]:
# I wasn't actually interested in the image links as image links, but rather to parse out product category information, 
# which didn't seem to be present anywhere else on the page. I created a function that uses urllib.parse from urlparse
# to grab the information that I wanted. I was having trouble getting re to work here, and this was simpler.

def extract_category(link_url):
    parsed_url = urlparse(link_url)
    path_components = parsed_url.path.split('/')
    category = '/'.join(path_components[3:4])
    return category

In [8]:
#creating the dataframe, creating a new column that calculates the clearance prices, applying my extract_category
clearance_df = pd.DataFrame(product_data)
clearance_df['Percentage Off'] = round(((clearance_df['Original Price'] - clearance_df['Clearance Price']) / clearance_df['Original Price']) * 100)
clearance_df['Product Category'] = clearance_df['Image Link'].apply(extract_category)

In [9]:
# HOT-SALE scraping. Code is nearly idential to the above.
# scraping the sale section for additional data

base_url = "https://www.canadiantire.ca/en/promotions/hot-sale.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver2 = webdriver.Chrome(service=service)

driver2.get(base_url)
time.sleep(5)
html_content = driver2.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver2.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver2.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver2.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 1884
Total number of pages: 79


In [10]:
sale_df = pd.DataFrame(sale_product_data)
sale_df['Percentage Off'] = round(((sale_df['Original Price'] - sale_df['Sale Price']) / sale_df['Original Price']) * 100)
sale_df['Product Category'] = sale_df['Image Link'].apply(extract_category)


In [11]:
#RED-ALERT-DEALS scraping. Code is nearly idential to the above.
# scraping the sale section for additional data

base_url = "https://www.canadiantire.ca/en/promotions/red-alert-deals.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver3 = webdriver.Chrome(service=service)

driver3.get(base_url)
time.sleep(5)
html_content = driver3.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data_2 = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver3.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver3, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver3.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info2 = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info2:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data_2.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver3.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 497
Total number of pages: 21


In [12]:
sale_df2 = pd.DataFrame(sale_product_data_2)
sale_df2['Percentage Off'] = round(((sale_df2['Original Price'] - sale_df2['Sale Price']) / sale_df2['Original Price']) * 100)
sale_df2['Product Category'] = sale_df2['Image Link'].apply(extract_category)

In [13]:
#checking
print(len(clearance_df))
clearance_df.sample(5)

1146


,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
1043,"Zombie Hunter Decal Kit, 6-in x 8-in",7.99,1.93,0.0,141-0580-2,https://www.canadiantire.ca/en/pdp/zombie-hunt...,https://media-www.canadiantire.ca/product/auto...,76.0,car-care-accessories
642,"AutoTrends Sporty Seat Cover, Black & Dark Red",59.99,39.93,2.5,032-6379-8,https://www.canadiantire.ca/en/pdp/autotrends-...,https://media-www.canadiantire.ca/product/auto...,33.0,car-care-accessories
461,SIMONIZ Platinum Long Handle Wool Brush,9.99,1.93,2.8,039-9088-8,https://www.canadiantire.ca/en/pdp/simoniz-pla...,https://media-www.canadiantire.ca/product/auto...,81.0,car-care-accessories
78,Body Glove Infant Nylon PFD/Life Jacket,59.99,29.93,3.1,079-2022-0,https://www.canadiantire.ca/en/pdp/body-glove-...,https://media-www.canadiantire.ca/product/play...,50.0,seasonal-recreation
1030,"Testors Craft Acrylic Paint, Fluorescent Orang...",5.99,0.43,4.8,048-8029-4,https://www.canadiantire.ca/en/pdp/testors-cra...,https://media-www.canadiantire.ca/product/fixi...,93.0,paint


In [14]:
#checking
print(len(sale_df))
sale_df.sample(5)

1884


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
1593,"Pilot 7-in Black Chrome Exhaust Tip, PM-1022",41.99,35.69,3.2,041-1239-0,https://www.canadiantire.ca/en/pdp/pilot-7-in-...,https://media-www.canadiantire.ca/product/auto...,15.0,heavy-auto-parts
911,"Nextbase 222 Dash Camera, Charging Cable, Moti...",149.99,129.99,4.7,035-0721-2,https://www.canadiantire.ca/en/pdp/nextbase-22...,https://media-www.canadiantire.ca/product/auto...,13.0,car-care-accessories
741,MAXIMUM M42 Bi-Metal Cobalt Hole Saw Set for W...,149.99,44.99,4.6,054-0315-4,https://www.canadiantire.ca/en/pdp/maximum-m42...,https://media-www.canadiantire.ca/product/fixi...,70.0,tools
1014,CANVAS Hanover Media Electric Fireplace TV Sta...,999.99,549.99,4.3,064-4016-8,https://www.canadiantire.ca/en/pdp/canvas-hano...,https://media-www.canadiantire.ca/product/livi...,45.0,home-decor
256,"CANVAS Hubbard 2-Tier Shoe Storage Cabinet, White",119.99,99.99,3.6,168-0146-6,https://www.canadiantire.ca/en/pdp/canvas-hubb...,https://media-www.canadiantire.ca/product/livi...,17.0,home-decor


In [15]:
#checking
print(len(sale_df2))
sale_df2.sample(5)

497


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
2,Keurig Tim Hortons Dark Roast K-Cup® Coffee Po...,35.99,32.99,4.3,153-2401-0,https://www.canadiantire.ca/en/pdp/keurig-tim-...,https://media-www.canadiantire.ca/product/livi...,8.0,food-drink
23,PADERNO Laurier 20pc Stainless Steel Flatware ...,129.99,49.99,4.8,142-2871-6,https://www.canadiantire.ca/en/pdp/paderno-lau...,https://media-www.canadiantire.ca/product/livi...,62.0,kitchen
236,Pine-Sol Spring Blossom Multi-Surface Cleaner ...,6.49,5.84,4.1,153-0512-8,https://www.canadiantire.ca/en/pdp/pine-sol-sp...,https://media-www.canadiantire.ca/product/livi...,10.0,cleaning
460,Scotts Ecosense Weed B Gon MAX Ready-To-Use We...,39.99,36.99,2.6,159-0056-2,https://www.canadiantire.ca/en/pdp/scotts-ecos...,https://media-www.canadiantire.ca/product/seas...,8.0,gardening
94,KitchenAid Stainless Steel Boxed Grater with B...,29.99,14.99,4.6,042-8549-8,https://www.canadiantire.ca/en/pdp/kitchenaid-...,https://media-www.canadiantire.ca/product/livi...,50.0,kitchen


In [16]:
#checking
clearance_df.dtypes

Product Name         object
Original Price      float64
Clearance Price     float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [17]:
#checking
sale_df.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [18]:
#checking
sale_df2.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [19]:
#no longer needed, I just used them to parse out the product type
clearance_df.drop('Image Link', axis=1, inplace=True)
sale_df.drop('Image Link', axis=1, inplace=True)
sale_df2.drop('Image Link', axis=1, inplace=True)

In [20]:
#reordering columns so they make more sense to me. 
clearance_column_order = ['Product Name',
                    'Original Price',
                    'Clearance Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

sale_column_order = ['Product Name',
                    'Original Price',
                    'Sale Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

clearance_df = clearance_df.loc[:, clearance_column_order]
sale_df = sale_df.loc[:, sale_column_order]
sale_df2 = sale_df2.loc[:, sale_column_order]

In [21]:
#change astype in case I want to use this later
clearance_df['Rating'] = clearance_df['Rating'].astype(float)
sale_df['Rating'] = sale_df['Rating'].astype(float)
sale_df2['Rating'] = sale_df2['Rating'].astype(float)

In [22]:
#checking
clearance_df.sample(5)

,Product Name,Original Price,Clearance Price,Percentage Off,Product Category,Product Code,Rating,Link
987,3-in and 4-in Green Grate,6.99,0.93,87.0,plumbing,063-2807-0,0.0,https://www.canadiantire.ca/en/pdp/3-in-and-4-...
844,Butler Creek Highlander Bone Collector Sling w...,48.99,17.93,63.0,hunting,075-8062-6,5.0,https://www.canadiantire.ca/en/pdp/butler-cree...
0,Bluehive 4-port Power Station,51.99,12.93,75.0,car-care-accessories,035-5304-6,4.0,https://www.canadiantire.ca/en/pdp/bluehive-4-...
66,Philips Sonicare Essential Clean Rechargeable ...,89.99,39.93,56.0,personal-garment-care,043-8685-0,4.5,https://www.canadiantire.ca/en/pdp/philips-son...
724,Hot Shot Rabbit Urine Predator Attractant,15.99,2.93,82.0,hunting,175-4377-4,3.7,https://www.canadiantire.ca/en/pdp/hot-shot-ra...


In [23]:
#checking
sale_df.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
1436,Purina Fancy Feast® Gravy Lovers Chicken Feast...,0.99,0.90,9.0,pet-care,142-0283-8,4.8,https://www.canadiantire.ca/en/pdp/purina-fanc...
1491,"MAXIMUM Hex Key Set, SAE, 9-pc",34.99,24.99,29.0,tools,058-9786-0,4.7,https://www.canadiantire.ca/en/pdp/maximum-hex...
1600,MotoMaster Clamping Handheld LED Work Light,59.99,29.99,50.0,car-care-accessories,037-6391-6,4.4,https://www.canadiantire.ca/en/pdp/motomaster-...
1705,"Preston Towel Bar, 24-in",41.99,33.59,20.0,plumbing,063-5163-2,3.3,https://www.canadiantire.ca/en/pdp/preston-tow...
354,Type A Alpha Under-Sink Metal Storage Organize...,46.99,24.99,47.0,home-organization,142-5507-8,4.2,https://www.canadiantire.ca/en/pdp/type-a-alph...


In [24]:
#checking
sale_df2.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
128,"Certified Light Duty Poly Tarp, Waterproof, 9-...",9.99,5.99,40.0,automotive-outdoor-adventure,040-5030-2,4.3,https://www.canadiantire.ca/en/pdp/certified-l...
479,"Purina® Fancy Feast® Cat Food, Grilled Chicken...",0.99,0.90,9.0,pet-care,042-6472-0,4.7,https://www.canadiantire.ca/en/pdp/purina-fanc...
376,SIMONIZ Platinum Carpet & Upholstery with Oxyg...,14.99,12.74,15.0,car-care-accessories,039-0570-2,3.5,https://www.canadiantire.ca/en/pdp/simoniz-pla...
83,"Heritage The Rock Frying Pan Non-stick, Dishwa...",119.99,24.99,79.0,kitchen,142-7072-8,4.6,https://www.canadiantire.ca/en/pdp/heritage-th...
98,"Lagostina Ceramic Bakeware Set, Red, Assorted ...",99.99,29.99,70.0,kitchen,142-8835-8,4.2,https://www.canadiantire.ca/en/pdp/lagostina-c...


In [25]:
#Removing characters that were causing sql import issues. 

clearance_df["Product Name"] = clearance_df["Product Name"].str.replace("'", '')
clearance_df["Product Name"] = clearance_df["Product Name"].str.replace("Â®", '')
sale_df["Product Name"] = sale_df["Product Name"].str.replace("'", '')
sale_df["Product Name"] = sale_df["Product Name"].str.replace("Â®", '')
sale_df2["Product Name"] = sale_df2["Product Name"].str.replace("'", '')
sale_df2["Product Name"] = sale_df2["Product Name"].str.replace("Â®", '')

In [38]:
clearance_df.to_csv("./data/clearance.csv", index=False, mode='w')
sale_df.to_csv("./data/sale.csv", index=False, mode='w')
sale_df2.to_csv("./data/sale2.csv", index=False, mode='w')

In [40]:
#just in case I only want to work with all the sale products, not seperated by specific promotion
allsales_df = pd.concat([sale_df, sale_df2], ignore_index=True)
allsales_df.to_csv("./data/allsales.csv", index=False, mode='w')

allproducts_df = pd.concat([clearance_df, sale_df, sale_df2], ignore_index=True)
allproducts_df.to_csv("./data/allproducts_df.csv", index=False, mode='w')

In [28]:
# I was thinking about saving the files with a datatime in case I wanted to do historical analysis at a later date. 
current_date = datetime.now().strftime("%Y-%m-%d")

clearance_df.to_csv(f"./data/clearance_{current_date}.csv", index=False)
sale_df2.to_csv(f"./data/sale2_{current_date}.csv", index=False)
allsales_df.to_csv(f"./data/allsales_{current_date}.csv", index=False)
allproducts_df.to_csv(f"./data/allproducts_{current_date}.csv", index=False)

In [46]:
# Connection details for uploading to the server
# Get the database connection details from environment variables
username = os.environ.get('DB_USERNAME')
password = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOST')
database = os.environ.get('DB_NAME')

# Create the database connection URL
# Create the URL to connect to the PostgreSQL database using the connection details
url = f'postgresql://{username}:{password}@{host}/{database}'

# Create the database engine
engine = create_engine(url)

# Create the metadata for defining the tables
metadata = MetaData()

# Define the "clearance" table with SQLAlchemy
# Specify the columns and their data types for the "clearance" table
clearance_table = Table('clearance', metadata,
    Column('product_name', String),
    Column('original_price', Float),
    Column('clearance_price', Float),
    Column('percentage_off', Float),
    Column('product_category', String),
    Column('product_code', String),
    Column('rating', Float),
    Column('link', String),
)

# Define the "sales" table with SQLAlchemy
# Specify the columns and their data types for the "sales" table
sales_table = Table('sales', metadata,
    Column('product_name', String),
    Column('original_price', Float),
    Column('clearance_price', Float),
    Column('percentage_off', Float),
    Column('product_category', String),
    Column('product_code', String),
    Column('rating', Float),
    Column('link', String),
)

# Create the tables in the database if they don't exist
metadata.create_all(engine)

# Delete all rows from the "clearance" table before inserting new data so that we only have fresh deals.
# Execute the DELETE statement to clear all rows from the "clearance" table
engine.execute(clearance_table.delete())

# Read clearance.csv and upload the data to the "clearance" table
# Convert the CSV data into a list of dictionaries to be inserted into the table
# Use None for empty fields in the database
with open('data/clearance.csv', 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    next(csv_reader)  # Skip the header row

    data_to_insert = []
    for row in csv_reader:
        original_price = float(row[1]) if row[1] != '' else None
        clearance_price = float(row[2]) if row[2] != '' else None
        percentage_off = float(row[3]) if row[3] != '' else None

        data_to_insert.append({
            "product_name": row[0],
            "original_price": original_price,
            "clearance_price": clearance_price,
            "percentage_off": percentage_off,
            "product_category": row[4],
            "product_code": row[5],
            "rating": float(row[6]) if row[6] != '' else None,
            "link": row[7]
        })

    # Insert the data into the "clearance" table
    insert_query = clearance_table.insert()
    engine.execute(insert_query, data_to_insert)

# Delete all rows from the "sales" table before inserting new data
# Execute the DELETE statement to clear all rows from the "sales" table
engine.execute(sales_table.delete())

# Read allsales.csv and upload the data to the "sales" table
# Convert the CSV data into a list of dictionaries to be inserted into the table
# Use None for empty fields in the database
with open('data/allsales.csv', 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    next(csv_reader)  # Skip the header row

    sales_data_to_insert = []
    for row in csv_reader:
        original_price = float(row[1]) if row[1] != '' else None
        clearance_price = float(row[2]) if row[2] != '' else None
        percentage_off = float(row[3]) if row[3] != '' else None

        sales_data_to_insert.append({
            "product_name": row[0],
            "original_price": original_price,
            "clearance_price": clearance_price,
            "percentage_off": percentage_off,
            "product_category": row[4],
            "product_code": row[5],
            "rating": float(row[6]) if row[6] != '' else None,
            "link": row[7]
        })

    # Insert the data into the "sales" table
    sales_insert_query = sales_table.insert()
    engine.execute(sales_insert_query, sales_data_to_insert)
    engine.dispose()

In [ ]:
## option for using the dataframes directly instead of CSVs to push to the SQL Server.

# username = os.environ.get('DB_USERNAME')
# password = os.environ.get('DB_PASSWORD')
# host = os.environ.get('DB_HOST')
# database = os.environ.get('DB_NAME')

# url = f'postgresql://{username}:{password}@{host}/{database}'
# engine = create_engine(url)
# metadata = MetaData()

# clearance_table = Table('clearance', metadata,
#     Column('product_name', String),
#     Column('original_price', Float),
#     Column('clearance_price', Float),
#     Column('percentage_off', Float),
#     Column('product_category', String),
#     Column('product_code', String),
#     Column('rating', Float),
#     Column('link', String),
# )

# sales_table = Table('sales', metadata,
#     Column('product_name', String),
#     Column('original_price', Float),
#     Column('clearance_price', Float),
#     Column('percentage_off', Float),
#     Column('product_category', String),
#     Column('product_code', String),
#     Column('rating', Float),
#     Column('link', String),
# )

# metadata.create_all(engine)

# clearance_data_to_insert = clearance_df.to_dict(orient='records')
# sales_data_to_insert = allsales_df.to_dict(orient='records')

# with engine.connect() as conn:
#     # Delete all rows from the "clearance" table before inserting new data
#     conn.execute(clearance_table.delete())

#     # Insert the data into the "clearance" table
#     conn.execute(clearance_table.insert(), clearance_data_to_insert)

#     # Delete all rows from the "sales" table before inserting new data
#     conn.execute(sales_table.delete())

#     # Insert the data into the "sales" table
#     conn.execute(sales_table.insert(), sales_data_to_insert)

# # Dispose the engine to release resources
# engine.dispose()